In [1]:
%%capture
!pip install openai

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 73.6/73.6 kB 6.2 MB/s eta 0:00:00


In [2]:
%%capture
import requests
import json
import ast
from google.colab import files
import pandas as pd
import io
import asyncio
import numpy as np
from sklearn.metrics import f1_score

In [3]:
def chat_completion(api_key, model, role, content, name=None,
                    temperature=1, top_p=1, n=1, stream=False, stop=None, max_tokens=None,
                    presence_penalty=0, frequency_penalty=0, logit_bias=None, user=None):
    url = "https://api.openai.com/v1/chat/completions"

    headers = {
        "Content-Type": "application/json",
        "Authorization": f"Bearer {api_key}"
    }

    data = {
        "model": model,
        "messages": [{"role": role,
                    "content": content,}],
    }

    if name is not None:
        data["name"] = name
    data["temperature"] = temperature
    data["top_p"] = top_p
    data["n"] = n
    data["stream"] = stream
    if stop is not None:
        data["stop"] = stop
    if max_tokens is not None:
        data["max_tokens"] = max_tokens
    data["presence_penalty"] = presence_penalty
    data["frequency_penalty"] = frequency_penalty
    if logit_bias is not None:
        data["logit_bias"] = logit_bias
    if user is not None:
        data["user"] = user

    response = requests.post(url, headers=headers, data=json.dumps(data))

    return response.json()

**Methods for classifying the abstract**

In [4]:
def check_bool_list(lst):
    return all(isinstance(i, bool) for i in lst)

def classify_abstract(abstract):
  openai_key = ''

  prompt = (f'Given the abstract :  {abstract}'
  + 'For the given abstract check each criteria whether they are true of false:'
  + '1. "The research question of the study is designed to quantify and statistically measure outcomes, correlations or differences between variables.",'
  + '2. "The data collected in the study is numerical and quantifiable.",'
  + '3. "The study applies statistical methods to analyze the data collected.",'
  + '4. "The data collection process uses (semi-)structured methods, for instance, surveys or questionnaires.",'
  + '5. "The findings are mainly presented in a numerical form, such as tables, graphs or measurements."'
  + '6. "The research question of the study is designed to explore, interpret or generate understanding about a phenomenon.",'
  + '7. "The data collected is non-numerical, for instance, text video or audio.",'
  + '8. "The study applies interpretive or subjective methodologies for data analyses.",'
  + '9. "The data collection process uses unstructured or semi-structured methods, for example, interviews, observations or analysis of documents.",'
  + '10. "The findings are presented in a narrative or descriptive form, providing detailed insights."'
  + 'Output should only have the following format, it should be two arrays containing true or false like:'
  + '[1, 2, 3, 4, 5] and [6, 7, 8, 9, 10] where the number represent the criteria number.')

  output = chat_completion(openai_key, 'gpt-3.5-turbo', 'user', prompt)

  try:
    if 'choices' in output:
      criteria_labels = output['choices'][0]['message']['content']

      criteria_labels = '[' + criteria_labels + ']'
      criteria_labels = criteria_labels.replace(' and ', ', ')
      criteria_labels = criteria_labels.replace("true", "True").replace("false", "False")

      parsed_labels = ast.literal_eval(criteria_labels)
    else:
      return classify_abstract(abstract)
  except:
    return classify_abstract(abstract)

  quantitative_labels = parsed_labels[0]
  if not check_bool_list(quantitative_labels):
    return classify_abstract(abstract)

  qualitative_labels = parsed_labels[1]
  if not check_bool_list(qualitative_labels):
    return classify_abstract(abstract)

  print("Quantitative Labels:", quantitative_labels)
  print("Qualitative Labels:", qualitative_labels)

  return sum(quantitative_labels), sum(qualitative_labels)

**Upload the test data**

In [6]:
uploaded_test_data = files.upload()
studies_df_test = pd.read_csv(io.BytesIO(uploaded_test_data['studies_test.csv']))

Saving studies_test.csv to studies_test.csv


**Run the model**

In [13]:
async def classify_abstract_concurrent(x):
    result = await asyncio.get_running_loop().run_in_executor(None, classify_abstract, x)
    return pd.Series(result, index=['quant_sum', 'qual_sum'])

async def process_rows():
    tasks = []
    for x in studies_df_test['combined']:
        tasks.append(classify_abstract_concurrent(x))
    results = await asyncio.gather(*tasks)
    return results

def run_asyncio(func):
    """Create a new event loop and run a function inside it."""
    import nest_asyncio
    nest_asyncio.apply()
    loop = asyncio.new_event_loop()
    asyncio.set_event_loop(loop)
    return loop.run_until_complete(func)

# Combine
studies_df_test['combined'] = studies_df_test['title'] + ' ' + studies_df_test['auth_kw'] + ' ' + studies_df_test['index_kw'] + ' ' + studies_df_test['abstract']

# Run the asyncio event loop and get the results
results = run_asyncio(process_rows())

# Create a new DataFrame with the results
results_df = pd.DataFrame(results)

# Combine the results DataFrame with the original DataFrame
studies_df_test = pd.concat([studies_df_test, results_df], axis=1)

# Display the updated DataFrame
print(studies_df_test)

Quantitative Labels: [False, True, True, True, True]
Qualitative Labels: [True, False, False, True, True]
Quantitative Labels: [False, True, True, True, True]
Qualitative Labels: [True, False, False, True, True]
Quantitative Labels: [True, True, True, True, True]
Qualitative Labels: [False, False, False, False, False]
Quantitative Labels: [True, True, True, True, True]
Qualitative Labels: [False, False, False, False, False]
Quantitative Labels: [True, True, True, True, False]
Qualitative Labels: [True, False, False, False, True]
Quantitative Labels: [False, False, False, True, True]
Qualitative Labels: [True, True, True, True, True]
Quantitative Labels: [True, True, True, True, True]
Qualitative Labels: [False, False, False, False, False]
Quantitative Labels: [False, True, True, True, True]
Qualitative Labels: [True, False, False, True, True]
Quantitative Labels: [False, False, True, False, True]
Qualitative Labels: [True, True, True, False, True]
Quantitative Labels: [True, True, True

**Evaluate the model**

In [14]:
# Calculate the percentiles
dif =  studies_df_test['quant_sum'] - studies_df_test['qual_sum']

q1 = np.percentile(dif, 33.33)
q2 = np.percentile(dif, 66.67)

# Divide into the correct predictions
studies_df_test['predicted'] = np.where((studies_df_test['quant_sum'].sub(studies_df_test['qual_sum']) < q1), 'Qualitative', np.where((studies_df_test['quant_sum'].sub(studies_df_test['qual_sum']) >= q1) & (studies_df_test['quant_sum'].sub(studies_df_test['qual_sum']) < q2), 'Mixed', 'Quantitative'))

# Calculate the accuracy
accuracy = (studies_df_test['method'] == studies_df_test['predicted']).mean() * 100

# Display the accuracy
print("Accuracy: {:.2f}%".format(accuracy))

# Calculate the F1 score
f1 = f1_score(studies_df_test['method'], studies_df_test['predicted'], average='weighted')

# Display the F1 score
print("F1 Score: {:.4f}".format(f1))

Accuracy: 43.48%
F1 Score: 0.2635
